# Preprocessing and geometry in a feature space

This coursework is designed to practically explore the content given in the lectures in week 3.

## Hints

- This one is a lot more challenging than last week so don't panic, break everything down into small steps

- For the further work questions it may be helpful to write your code using functions rather than scripts (or even classes if they are familiar to you)

In [197]:
import numpy as np
#DO NOT CHANGE THIS CODE
X_train=np.loadtxt('./data/X_train_linear.txt')
X_test=np.loadtxt('./data/X_test_linear.txt')
y_train=np.loadtxt('./data/y_train_linear.txt')
y_test=np.loadtxt('./data/y_test_linear.txt')

## 1) Geometry in a kernel-defined feature space

### a) Compute the linear kernel of the linear training data without centering the data and calculate the norm of the mean of a sample

In [198]:
from sklearn.metrics.pairwise import linear_kernel

K=linear_kernel(X_train)

print(f"In original space: {np.linalg.norm(X_train.mean(axis=0))}")

m=X_train.shape[0]
j=np.ones((m,1))
mean_of_a_sample=np.sqrt(j.T@K@j)/m

print(f"In kernel space: {mean_of_a_sample}")

In original space: 0.6018046905945809
In kernel space: [[0.60180469]]


As expected the mean of the sample can be calculated in both the original ....

### b) Calculate the expected squared distance to the mean of a sample

In [199]:


print(f"Expected squared distance in kernel feature space: {np.trace(K)/m-j.T@K@j/m**2}")

#(np.linalg.norm(X_train - X_train.mean(axis=0),axis=1)**2).mean()
#distance = X_train-sample_mean


Expected squared distance in kernel feature space: [[19.23293439]]


### c) By subtracting the appropriate matrix, implement the centering of this kernel matrix and verify that the norm of the mean of a sample is zero

In [200]:
m=X_train.shape[0]
j=np.ones((m,1))
K_hat=K-1/m*(j@j.T@K+K@j@j.T)+1/m**2*(j.T@K@j)*j@j.T
print(f"mean of a sample in centred kernel: {(j.T@K_hat@j)/m**2}")

mean of a sample in centred kernel: [[-1.94527949e-16]]


### d) Verify that the expected squared distance to the mean of a sample with the centred kernel is the same as the uncentred kernel

In [201]:
print(f"Expected squared distance in kernel feature space: {np.trace(K_hat)/m-j.T@K_hat@j/m**2}")

Expected squared distance in kernel feature space: [[19.23293439]]


### e) By centering the original data and computing the linear kernel representation of the centered data show that the kernel representations are equivalent

In [202]:
np.max(np.abs(linear_kernel(X_train-X_train.mean(axis=0))-K_hat))

1.4210854715202004e-14

## 2) Simple Classification Algorithm
In this part we will build the simple classifier described in the lectures

In [203]:
import numpy as np
#DO NOT CHANGE THIS CODE
X_train=np.loadtxt('./data/X_train_poly.txt')
X_test=np.loadtxt('./data/X_test_poly.txt')
y_train=np.loadtxt('./data/y_train_poly.txt')
y_test=np.loadtxt('./data/y_test_poly.txt')

### a) Write a function which calculates the mean of the kernel function of a sample with the positive and negative training data (i.e. the terms that aren't b)

In [204]:
def get_m(y):
    m_pos=y[y>0].shape[0]
    m_neg=y[y<0].shape[0]
    return m_pos, m_neg

def kernel_function(K,y):
    m_pos,m_neg=get_m(y)
    positive_mean=K[:,y>0].sum(axis=1)/m_pos
    negative_mean=K[:,y<0].sum(axis=1)/m_neg
    return positive_mean-negative_mean

In [205]:
K=linear_kernel(X_train)
get_m(y_train)
kernel_function(K,y_train)

array([ 6.25609520e-01, -2.32656312e+00,  7.60642583e-01, -5.80356108e-01,
       -1.48849059e+00, -1.86896475e+00, -2.08447008e+00, -1.71472451e-01,
       -2.72415073e+00,  9.51467627e-01, -2.98943905e-01, -1.22342750e+00,
       -2.01697207e-01, -3.28055737e-01, -5.80513275e-01,  2.20288617e+00,
       -2.37536560e-01,  2.85227617e-02, -4.48560416e-01, -3.78632030e-02,
        1.75133072e+00,  2.22555739e+00,  8.68513078e-01,  2.02140156e+00,
        1.98771729e+00,  1.94415532e+00, -1.12283622e+00, -1.35696469e+00,
       -5.49945993e-01, -2.07905923e+00, -1.80405842e+00, -6.55863262e-02,
        1.97259319e+00,  8.32757865e-02, -1.64279493e+00, -4.97748307e-01,
       -1.43308834e+00, -2.67113416e+00,  1.03157183e+00, -1.31540116e+00,
        1.46732812e+00, -7.76615082e-02,  6.27688534e-01, -2.93553951e+00,
       -1.29553608e+00,  3.44035866e+00,  2.31744131e+00, -1.74318130e-01,
        2.77729299e+00, -2.49241431e-01,  1.51421362e+00, -6.97346386e-01,
       -9.38259052e-01, -

### b) Write a function which calculates the b term in the simple classifier

In [206]:
def calculate_b(K,y):
    m_pos,m_neg=get_m(y)
    return K[y>0,y>0].sum()/m_pos**2-K[y<0,y<0].sum()/m_neg**2

### c) Write the decision function of the classifier

In [207]:
def simple_classifier(K_test,K_train,y_train):
    return np.sign(kernel_function(K_test,y_train)-calculate_b(K_train,y_train))

### c) Calculate the accuracy of this simple classifier

In [208]:
from sklearn.metrics import accuracy_score

K_train=linear_kernel(X_train)
K_test=linear_kernel(X_test,X_train)

y_pred_train=simple_classifier(K_train,K_train,y_train)
y_pred_test=simple_classifier(K_test,K_train, y_train)

print(f"Train accruacy: {accuracy_score(y_train,y_pred_train)}")
print(f"Test accruacy: {accuracy_score(y_test,y_pred_test)}")

Train accruacy: 0.82
Test accruacy: 0.808


## 3) Fisher Discriminant Analysis

### a) Write a function which calculates the matrices B, C, and D

In [209]:
def calculate_C(y):
    m_pos,m_neg=get_m(y)
    m=m_pos+m_neg
    positive_entry=2*m_neg/(m*m_pos)
    negative_entry=2*m_pos/(m*m_neg)
    C=np.zeros((m,m))
    C[y>0,y>0]=positive_entry
    C[y<0,y<0]=negative_entry
    return C

def calculate_D(y):
    m_pos,m_neg=get_m(y)
    m=m_pos+m_neg
    diag=np.zeros(m)
    diag[y>0]=2*m_neg/m
    diag[y<0]=2*m_pos/m
    D=np.diag(diag)
    return D

def calculate_B(y):
    D=calculate_D(y)
    C=calculate_C(y)
    return D-C

### b) Write a function which calculates the dual variables for the FDA classifier

In [210]:
def get_alpha(K,y,lam=0.001):
    B=calculate_B(y)
    return np.linalg.inv(B@K+lam*np.eye(K.shape[0]))@y

### c) Write a function which calculates t and therefore the offset term b

In [211]:
def get_t(y):
    m_pos,m_neg=get_m(y)
    t=np.ones_like(y)
    t[y>0]=1/m_pos
    t[y<0]=1/m_neg
    return t

def get_b(alpha,K,t):
    return 0.5*alpha@K@t

### d) Write the decision function of the classifier

In [212]:
def fda(K_test,K_train,y_train,lam=1000):
    alpha=get_alpha(K_train,y_train,lam=lam)
    t=get_t(y_train)
    b=get_b(alpha,K_train,t)
    return np.sign(K_test@alpha-b)

## 4) Further Investigation

### a) For one or both classifiers Demonstrate how the performance of the classifier changes if one class is overrepresented in the training data as compared to the test data (i.e. there are imbalanced classes)

In [219]:
from sklearn.metrics.pairwise import polynomial_kernel

K_train=polynomial_kernel(X_train,degree=6)
K_test=polynomial_kernel(X_test,X_train,degree=6)

y_pred_train=fda(K_train,K_train,y_train, lam=0.0000000001)
y_pred_test=fda(K_test,K_train, y_train,lam=0.00000001)

print(f"Train accruacy: {accuracy_score(y_train,y_pred_train)}")
print(f"Test accruacy: {accuracy_score(y_test,y_pred_test)}")

Train accruacy: 1.0
Test accruacy: 0.608


### b) Demonstrate how regularisation can be used to improve the generalisation of the FDA classifier